<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_INTEGRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env --upgrade -q
!pip install --upgrade openai httpx==0.27.2 -q # Downgrade httpx to a compatible version

In [ ]:
import openai
import torch
import torch.nn as nn
import os
import colab_env

from openai import OpenAI
openai.api_key = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

# Function to get GPT-3.5 output using the new client-based API
def get_gpt35_response(prompt):
  client = openai.OpenAI()  # Initialize the OpenAI client
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return response.choices[0].message.content.strip()

In [20]:
import os
from openai import OpenAI

# Initialize the OpenAI client
openai.api_key = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


class MemoryExpert:
    """
    Represents a memory expert with specialized knowledge and a b-tensor score.
    """

    def __init__(self, name, domain, knowledge_base, b_tensor):
        self.name = name  # Add a name for the expert
        self.domain = domain
        self.knowledge_base = knowledge_base
        self.b_tensor = b_tensor  # Add b-tensor score (dictionary)

    def query(self, question):
        """
        Answers a question based on the expert's knowledge (improved).
        """
        for key in self.knowledge_base:
            if any(word in question for word in key.split()):
                return self.knowledge_base[key]
        return "I don't know."


class LLMWrapper:
    """
    A wrapper for the GPT-4 language model.
    """

    def generate_text(self, prompt):
        """
        Generates text based on the prompt using GPT-4.
        """
        try:
            client = openai.OpenAI()
            response = client.chat.completions.create(
                model="gpt-4",  # Updated to use gpt-4
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message.content.strip()
        except openai.OpenAIError as e:
            print(f"Error generating text: {e}")
            return None


class AgenticAI:
    """
    An agent that uses an LLM and MoME to achieve goals.
    """

    def __init__(self, llm, memory_experts):
        self.llm = llm
        self.memory_experts = memory_experts

    def achieve_goal(self, goal):
        """
        Uses the LLM and memory experts to achieve a goal.
        """
        # 1. Generate initial plan using the LLM
        plan = self.llm.generate_text(f"How to achieve {goal}")
        print(f"Initial plan: {plan}")

        # 2. Consult memory experts for specific steps
        refined_plan = []
        steps = plan.split(";")  # Split the plan into steps
        for i, step in enumerate(steps):  # Use enumerate to get step index
            found_expert = False
            for expert in self.memory_experts:
                # Condition for applying expert advice (updated)
                if expert.domain in step and any(
                        keyword in step
                        for keyword in ["booking", "website", "airline", "flights"]  # More specific keywords
                ) and any(  # Check for contextual clues
                        clue in step
                        for clue in ["searching for flights", "comparing prices", "finding deals"]
                ):
                    expert_advice = expert.query(step)
                    refined_plan.append(
                        f"{step} ({expert_advice} - {expert.name}, FA: {expert.b_tensor.get('FA', 'N/A')}, MD: {expert.b_tensor.get('MD', 'N/A')})"
                    )  # Include expert name and b-tensor scores
                    found_expert = True
                    break
            if not found_expert:
                refined_plan.append(step)

        # 3. Execute the refined plan (in this case, just print)
        print(f"Refined plan: {'; '.join(refined_plan)}")
        print("\n")  # Add an extra newline for better readability


# Example usage with b-tensor scores
memory_experts = [
    MemoryExpert("Pilot Ace", "aviation", {
        "flight plan":
        "1. Check weather conditions; 2. File flight plan with FAA"
    }, {"FA": 0.8, "MD": 0.6}),
    MemoryExpert(
        "Coach Sport", "baseball", {
            "Yankees":  # Updated knowledge for attending games as a spectator
            "1. Purchase tickets in advance; 2. Take the 4 train to 161st Street; 3. Enjoy the atmosphere and the game"
        }, {"FA": 0.5, "MD": 0.7, "RD": 0.9}),
    MemoryExpert("Travel Guru", "travel", {
        "flight": "Use a flight booking website or a travel agent"
    }, {"FA": 0.7, "MD": 0.8}),
    MemoryExpert("NYC Navigator", "New York", {
        "NYC": "Consider traffic and transportation options"
    }, {"FA": 0.9, "MD": 0.5})
]

llm = LLMWrapper()
agent = AgenticAI(llm, memory_experts)

agent.achieve_goal(
    "create a flight plan from Montreal to NYC and watch a baseball game in the Yankees Stadium"
)

Initial plan: Here's a step-by-step guide on how to plan your trip from Montreal to NYC to watch a Yankees game:

1. **Flight Booking**: You can easily book your flights online from Montreal to NYC via various travel websites or directly from the airlines. You can choose to fly into any of New York's international airports - JFK, LaGuardia, or Newark. Consider booking a flight that would give you enough time to settle down and reach the stadium before the game starts.

2. **Accommodation**: Book accommodation for your stay in NYC. Try to find a hotel close to the Yankees Stadium or with easy access to public transport. Websites like Booking.com, Expedia, or Airbnb can help you find suitable accommodation.

3. **Transport Details to Stadium**: The Yankee Stadium is located in the Bronx and is accessible via public transportation. The B, D, or 4 subway lines will get you directly to the Stadium at 161st Street Station.

4. **Getting Yankees Game Tickets**: Visit the Yankees' official web